In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

In [ ]:
#Function to read the files line by line
def read_file(file_name):
    file = open(file_name,'r')
    filedata = file.readlines()
    article = filedata[0].split('. ')
    sentences = []
    for sentences in article:
        s = sentences.replace('[^a-zA-Z]',' ').split(' ')
        sentences = sentences + str(s)
    list(sentences).pop()
    return sentences
        
    
    

In [ ]:
#Function for sentence similarity
def sentence_similarity(sent1,sent2,stopwords=None):
    if stopwords is None:
        stopwords = []
        sent1 = [w.lower() for w in sent1]
        sent2 = [w.lower() for w in sent2]
        all_words = list(set(sent1+sent2)) #to get unique characters in both Sent1 and Sent2
        #Creating Vectors 
        vector1 = [0]*len(all_words)
        vector2 = [0]*len(all_words)
        for w in sent1:
            if w in stopwords:
                continue
            vector1[all_words.index(w)] += 1
        for w in sent2:
            if w in stopwords:
                continue
            vector2[all_words.index(w)] += 1
        return 1-cosine_distance(vector1,vector2)
        

In [ ]:
#Generating similarity matrix to find out the similar words
def gen_sim_matrix(sentences,stopwords):
    stop_words = stopwords.words('english')
    similarity_matrix = np.zeros((len(sentences),len(sentences)))
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
            similarity_matrix[idx1][idx2]=sentence_similarity(sentences[idx1],sentences[idx2],stop_words)
    return similarity_matrix    
    

In [ ]:
#generating the summary
from nltk.corpus import stopwords
def generate_summary(file_name,top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []
    sentences = read_file(file_name)
    sentence_similarity_matrix = gen_sim_matrix(sentences,stop_words)
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)),reverse=True)
    for i in range(top_n):
        summarize_text = summarize_text + ranked_sentences[i][1]
    print('Summary \n','. '.join(summarize_text))
        

    

In [ ]:
generate_summary('abc.story',2)